**Projeto predição de mutação gene ALDH2**

O seguinte projeto foi realizado anteriormente com dados reais obtidos a partir do projeto de pesquisa de doutorado da medicina/USP. Para manter a integridade e proteção dos daods da pesquisa, foram utilizados dados fictícios nesta análise.

**Importância da predição de indivíduos com mutação no gene ALDH2:**
A enzima mitocondrial ALDH2 desempenha um papel fundamental na metabolização do etanol. A presença de uma mutação genética no gene responsável por essa enzima resulta em uma diminuição de sua atividade enzimática, levando a um comprometimento do processo metabólico. Dadas as consideráveis despesas associadas à realização de análises de confirmação desta mutação, a adoção de abordagens de predição utilizando machine learning emerge como uma alternativa viável tanto para fins terapêuticos quanto para o avanço de
investigações futuras.

**Variáveis utilizadas e desfecho**
O desfecho utilizado foi a classificação de existência ou não de mutação no gene ALDH2. Para isso foram utilizados variáveis possivelmente preditoras de mutação do gene, sendo elas:
- Indíce de massa corporal (IMC)
- Percentual de massa magra
- Percentual de massa gorda
- Testes de força isométrica pré e pós dano muscular induzido por treino de força (MVIC 1-5)
- Teste de força máxima no leg-press (1RM leg-press)
- Frequência cardíaca pré, 30 e 60 minutos pós ingestão de álcool (FC)
- Pressão arterial sistólica pré, 30 e 60 minutos pós ingestão de álcool (PAS)
- Pressão arterial diastólica pré, 30 e 60 minutos pós ingestão de álcool
Etapas de Análise (PAD)
- Escala de dor muscular pós dano muscular induzido por treino de força 24 e 48h
- Autoavaliação de rubor para a pergunta "Você geralmente tem a tendência de ficar com a pele do resto vermelha imediatamente após o consumo de alguma dose de álcool?"
- Autoavaliação de rubor para a pergunta "Você geralmente tem a tendência de ficar com a pele do resto vermelha imediatamente após o seu primeiro ou segundo ano em que começou a consumir álcool?"
- Visualização de rubor após ingestão de álcool

**Roteiro:**

1. Instalação dos pacotes recomendados

2. Importando pacotes e denominando-os

3. Selecionando e verificando o banco de dados

4. Filtando banco de dados
  
  4.1 Transformando variáveis categóricas em label enconding/one-hot enconding
  
  4.2 Padronização variáveis quantitativas

5. Random Forest

  5.1 Random forest com hiperparâmetros

6. XGBOOST

  6.1 XGBOOST com hiperparâmetros

7. CATBOOST

  7.1 CATBOOST com hiperparâmetros

8. Variáveis selecionadas pelo algoritmo

  8.1 BORUTA

  8.2 Shap

1. Instação dos pacotes recomendados

In [ ]:
!pip install dfply >> /dev/null
!pip install xgboost >> /dev/null
!pip install boruta >> /dev/null
!pip install shap >> /dev/null
!pip install catboost >> /dev/null
!pip install --upgrade shap >> /dev/null
!pip install scikit-plot >> dev/null

2. Importando pacotes e denominando-os

In [ ]:
#Importação dos pacotes
import pandas as pd
import numpy as np
import lightgbm as lgb
import xgboost as xgb
import shap
from dfply import *
from sklearn import preprocessing
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, recall_score, precision_score
from sklearn.metrics import classification_report
from boruta import BorutaPy
from sklearn.metrics import make_scorer, f1_score
from catboost import CatBoostClassifier

np.random.seed(42)  # semente de aleatoriedade

3. Selecionando e verificando o banco de dados

In [ ]:
#Puxando o banco de dados
dataset = pd.read_excel ('https://drive.google.com/uc?export=download&id=1EgMqXPBioazgSim1m8D7Dk6D2f1_bBCv')

In [ ]:
dataset.shape

In [ ]:
dataset.head(5)

In [ ]:
dataset.info() #Obsevando o tipo das variáveis parametrizadas na chamada

4. Filtando banco de dados

Removendo as variáveis que poderiam ser vazamento de dados ou desnecessárias.

ID do voluntário
Idade
Massa e estatura (deixando apenas o IMC)

In [ ]:
ALDH2data = (dataset >>
            drop(X.ID, X.Idade, X.Massa, X.Estatura))

4.1 Imputando missing data

Nos missing data de variáveis quantitativas foi utilizado a média e nas variáveis categóricas a moda

In [ ]:
ALDH2data.describe().T #Observando dados estatísticos básicos

In [ ]:
#Lista de nomes das colunas quantitativas e categóricas
col_quant = ['IMC', 'Massa Magra', 'Massa Gorda', '1 Teste MVIC', '2 Teste MVIC', '3 Teste MVIC', '4 Teste MVIC', '5 Teste MVIC', '1RM leg-press', 'FC pré', 'FC 30 min', 'FC 60 min', 'PAS pré', 'PAS 30 min', 'PAS 60 min', 'PAD pré', 'PAD 30 min', 'PAD 60 min', 'Escala de Dor 24h', 'Escala de Dor 48h']
col_cat = ['Mutação', 'Autoavaliação rubor atual', 'Autoavaliação rubor inicial','Visualiação rubor']

#Dataframe separado para cada tipo de variável
ALDH2quant = ALDH2data[col_quant]
ALDH2cat = ALDH2data[col_cat]

In [ ]:
#Imputando a média nas variáveis quantitativas
imputerquant = SimpleImputer(strategy='mean')
imputerquant.fit(ALDH2quant)
ALDH2data_quant = imputerquant.transform(ALDH2quant)

In [ ]:
#Imputando a moda nas variáveis categóricas
imputercat = SimpleImputer(strategy='most_frequent')
imputercat.fit(ALDH2cat)
ALDH2data_cat = imputercat.transform(ALDH2cat)

In [ ]:
#Transformando os arrays NumPy em dataframes
ALDH2data_quant = pd.DataFrame(ALDH2data_quant, columns=ALDH2quant.columns)
ALDH2data_cat = pd.DataFrame(ALDH2data_cat, columns=ALDH2cat.columns)

#Juntando os dataframes quantitativos e categóricos ao longo das colunas (axis=1)
dfALDH2 = pd.concat([ALDH2data_quant, ALDH2data_cat], axis=1)

#Observando se foi imputado os valores missing
dfALDH2.info()

4.2 Transformando variáveis categóricas em label enconding

Não foi necessário one-hot encoding, pois todas as varíaveis categóricas só possuiam 2 categórias após imput dos missing data

In [ ]:
#Label enconding
dfALDH2[['Mutação','Autoavaliação rubor atual', 'Autoavaliação rubor inicial', 'Visualiação rubor']] = dfALDH2[['Mutação', 'Autoavaliação rubor atual', 'Autoavaliação rubor inicial', 'Visualiação rubor']].apply(preprocessing.LabelEncoder().fit_transform)

In [ ]:
dfALDH2.head(5)

4.3 Padronização variáveis quantitativas

In [ ]:
# Selecionar apenas as colunas com dados quantitativos
var_quant = dfALDH2.select_dtypes(include=['float64']).columns

#Padronização das variáveis quantitativas
scaler = StandardScaler()
dfALDH2[var_quant] = scaler.fit_transform(dfALDH2[var_quant])

5. Random Forest

Por conta do n amostral ser baixo foi realizado uma cross-validation direta, sem realização da separação entre treino e teste. O número de folds foi definido pelo n amostral ser =24, onde ambos os grupos (desfecho posítivo ou negativo) eram multiplos de 3.

In [ ]:
#Definindo as variáveis independentes e a variável dependente
X = dfALDH2.drop(columns=['Mutação'])
y = dfALDH2['Mutação'].values

In [ ]:
model = make_pipeline(StandardScaler(), RandomForestClassifier())

In [ ]:
#Definindo o número de folds
n_folds = 3

#Objeto de validação cruzada com 3 folds
kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)

#Realizando a validação cruzada e calculando as métricas
cross_val_scores = cross_val_score(model, X, y, cv=kf, scoring='accuracy')
cross_val_auc_scores = cross_val_score(model, X, y, cv=kf, scoring='roc_auc')
cross_val_precision_scores = cross_val_score(model, X, y, cv=kf, scoring='precision')
cross_val_recall_scores = cross_val_score(model, X, y, cv=kf, scoring='recall')

#Pontuações de validação cruzada para cada fold
for i, (accuracy, auc, precision, recall) in enumerate(zip(cross_val_scores, cross_val_auc_scores, cross_val_precision_scores, cross_val_recall_scores)):
    print(f'Fold {i + 1} - Acurácia: {accuracy:.4f}, AUC: {auc:.4f}, Precisão: {precision:.4f}, Recall: {recall:.4f}')

#Calculando as métricas médias em todos os folds
mean_accuracy = cross_val_scores.mean()
mean_auc = cross_val_auc_scores.mean()
mean_precision = cross_val_precision_scores.mean()
mean_recall = cross_val_recall_scores.mean()

print('Random Forest')
print(f'Acurácia Média: {mean_accuracy:.4f}')
print(f'AUC Médio: {mean_auc:.4f}')
print(f'Precisão Média: {mean_precision:.4f}')
print(f'Recall Médio: {mean_recall:.4f}')

In [ ]:
#Definindo a grade de hiperparâmetros
param_grid = {
    'n_estimators': [5, 10, 20],
    'max_depth': [3, 4, 5, None],
    'min_samples_split':[4, 5, 6],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2']
}

#Criando o modelo Random Forest
model = RandomForestClassifier(random_state=42)

#Criando o objeto de validação cruzada com 3 folds
kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)

#Criando o objeto GridSearchCV com scoring='roc_auc'
grid_search = GridSearchCV(model, param_grid, cv=kf, scoring='roc_auc', verbose=1, n_jobs=-1)

#Realizando o grid search
grid_search.fit(X, y)

#Obtendo os melhores hiperparâmetros encontrados
best_params = grid_search.best_params_
best_auc = grid_search.best_score_

# Imprimindo os melhores hiperparâmetros e o AUC correspondente
print("Melhores Hiperparâmetros:")
print(best_params)

5.1 Random Forest com os melhores hiperparâmetros

In [ ]:
# Definindo os melhores hiperparâmetros
best_params = {'max_depth': 3, 'max_features': 'sqrt', 'min_samples_leaf': 4, 'min_samples_split': 4, 'n_estimators': 20}

# Definindo o número de folds
n_folds = 3

# Criando o modelo Random Forest com os melhores hiperparâmetros
model = RandomForestClassifier(random_state=42, **best_params)

#Objeto de validação cruzada com 3 folds
kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)

#Realizando a validação cruzada e calculando as métricas
cross_val_scores = cross_val_score(model, X, y, cv=kf, scoring='accuracy')
cross_val_auc_scores = cross_val_score(model, X, y, cv=kf, scoring='roc_auc')
cross_val_precision_scores = cross_val_score(model, X, y, cv=kf, scoring='precision')
cross_val_recall_scores = cross_val_score(model, X, y, cv=kf, scoring='recall')

#Pontuações de validação cruzada para cada fold
for i, (accuracy, auc, precision, recall) in enumerate(zip(cross_val_scores, cross_val_auc_scores, cross_val_precision_scores, cross_val_recall_scores)):
    print(f'Fold {i + 1} - Acurácia: {accuracy:.4f}, AUC: {auc:.4f}, Precisão: {precision:.4f}, Recall: {recall:.4f}')

#Calculando as métricas médias em todos os folds
mean_accuracy = cross_val_scores.mean()
mean_auc = cross_val_auc_scores.mean()
mean_precision = cross_val_precision_scores.mean()
mean_recall = cross_val_recall_scores.mean()

# Imprimindo as métricas médias
print('Random Forest ajustado')
print(f'Acurácia Média: {mean_accuracy:.4f}')
print(f'AUC Médio: {mean_auc:.4f}')
print(f'Precisão Média: {mean_precision:.4f}')
print(f'Recall Médio: {mean_recall:.4f}')

6. XGBOOST

In [ ]:
#Definindo as variáveis independentes e a variável dependente
X = dfALDH2.drop(columns=['Mutação'])
y = dfALDH2['Mutação'].values.tolist()

In [ ]:
#Criando o modelo XGBoost
model = make_pipeline(StandardScaler(), XGBClassifier(random_state=42, objective='binary:logistic', use_label_encoder=False))

#Definindo o número de folds
n_folds = 3

#Objeto de validação cruzada com 3 folds
kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)

#Realizando a validação cruzada e calculando as métricas
cross_val_accuracies = cross_val_score(model, X, y, cv=kf, scoring='accuracy')
cross_val_auc_scores = cross_val_score(model, X, y, cv=kf, scoring='roc_auc')
cross_val_precision_scores = cross_val_score(model, X, y, cv=kf, scoring='precision')
cross_val_recall_scores = cross_val_score(model, X, y, cv=kf, scoring='recall')

#Pontuações de validação cruzada para cada fold
for i, (accuracy, auc, precision, recall) in enumerate(zip(cross_val_accuracies, cross_val_auc_scores, cross_val_precision_scores, cross_val_recall_scores)):
    print(f'Fold {i + 1} - Acurácia: {accuracy:.4f}, AUC: {auc:.4f}, Precisão: {precision:.4f}, Recall: {recall:.4f}')

#Calculando as métricas médias em todos os folds
mean_accuracy = cross_val_accuracies.mean()
mean_auc = cross_val_auc_scores.mean()
mean_precision = cross_val_precision_scores.mean()
mean_recall = cross_val_recall_scores.mean()

print('XGBOOST')
print(f'Acurácia Média: {mean_accuracy:.4f}')
print(f'AUC Médio: {mean_auc:.4f}')
print(f'Precisão Média: {mean_precision:.4f}')
print(f'Recall Médio: {mean_recall:.4f}')

In [ ]:
#Definindo lista de hiperparâmetros a serem otimizados
param_grid = {
    'xgbclassifier__n_estimators': [5, 10, 20],
    'xgbclassifier__max_depth': [3, 4, 5],
    'xgbclassifier__learning_rate': [0.01, 0.1, 0.2],
    'xgbclassifier__subsample': [0.8, 1.0],
    'xgbclassifier__colsample_bytree': [0.8, 1.0]
}

#Criando o objeto de validação cruzada com 3 folds
kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)

#Criando o objeto GridSearchCV com scoring='roc_auc'
grid_search = GridSearchCV(model, param_grid, cv=kf, scoring='roc_auc', verbose=1, n_jobs=-1)

#Realizando o grid search
grid_search.fit(X, y)

#Obtendo os melhores hiperparâmetros encontrados
best_params = grid_search.best_params_
best_auc = grid_search.best_score_

#Imprimindo os melhores hiperparâmetros e o AUC correspondente
print("Melhores Hiperparâmetros:")
print(best_params)
print(f'Melhor AUC: {best_auc:.4f}')

6.1 XGBOOST com melhores hiperparâmetros

In [ ]:
objective='binary:logistic',
    use_label_encoder=False
)

#Definindo o número de folds
n_folds = 3

#Realizando a validação cruzada e calculando as métricas
cross_val_auc_scores = cross_val_score(best_xgb_model, X, y, cv=kf, scoring='roc_auc')
cross_val_accuracy_scores = cross_val_score(best_xgb_model, X, y, cv=kf#Criando o modelo XGBoost com os melhores hiperparâmetros
best_xgb_model = XGBClassifier(
    colsample_bytree=0.8,
    learning_rate=0.1,
    max_depth=3,
    n_estimators=20,
    subsample=1.0,
    random_state=42,, scoring='accuracy')
cross_val_precision_scores = cross_val_score(best_xgb_model, X, y, cv=kf, scoring='precision')
cross_val_recall_scores = cross_val_score(best_xgb_model, X, y, cv=kf, scoring='recall')

#Pontuações de validação cruzada para cada fold
for i, (auc, accuracy, precision, recall) in enumerate(zip(cross_val_auc_scores, cross_val_accuracy_scores, cross_val_precision_scores, cross_val_recall_scores)):
    print(f'Fold {i + 1} - AUC: {auc:.4f}, Acurácia: {accuracy:.4f}, Precisão: {precision:.4f}, Recall: {recall:.4f}')

#Calculando as métricas médias em todos os folds
mean_auc = cross_val_auc_scores.mean()
mean_accuracy = cross_val_accuracy_scores.mean()
mean_precision = cross_val_precision_scores.mean()
mean_recall = cross_val_recall_scores.mean()

#Imprimindo as métricas médias
print('XGBOOST ajustado')
print(f'AUC Médio: {mean_auc:.4f}')
print(f'Acurácia Média: {mean_accuracy:.4f}')
print(f'Precisão Média: {mean_precision:.4f}')
print(f'Recall Médio: {mean_recall:.4f}')


7. CATBOOST

In [ ]:
#Definindo as variáveis independentes e a variável dependente
X = dfALDH2.drop(columns=['Mutação'])
y = dfALDH2['Mutação'].values

In [ ]:
X = np.array(X)
y = np.array(y)

In [ ]:
#Criando o modelo CatBoost
catboost_model = CatBoostClassifier(random_state=42, verbose=False)

#Definindo o número de folds
n_folds = 3

#Realizando a validação cruzada e calculando as métricas
cross_val_auc_scores = cross_val_score(catboost_model, X, y, cv=kf, scoring='roc_auc')
cross_val_accuracy_scores = cross_val_score(catboost_model, X, y, cv=kf, scoring='accuracy')
cross_val_precision_scores = cross_val_score(catboost_model, X, y, cv=kf, scoring='precision')
cross_val_recall_scores = cross_val_score(catboost_model, X, y, cv=kf, scoring='recall')

#Pontuações de validação cruzada para cada fold
for i, (auc, accuracy, precision, recall) in enumerate(zip(cross_val_auc_scores, cross_val_accuracy_scores, cross_val_precision_scores, cross_val_recall_scores)):
    print(f'Fold {i + 1} - AUC: {auc:.4f}, Acurácia: {accuracy:.4f}, Precisão: {precision:.4f}, Recall: {recall:.4f}')

#Calculando as métricas médias em todos os folds
mean_auc = cross_val_auc_scores.mean()
mean_accuracy = cross_val_accuracy_scores.mean()
mean_precision = cross_val_precision_scores.mean()
mean_recall = cross_val_recall_scores.mean()

#Imprimindo as métricas médias
print('CATBOOST')
print(f'AUC Médio: {mean_auc:.4f}')
print(f'Acurácia Média: {mean_accuracy:.4f}')
print(f'Precisão Média: {mean_precision:.4f}')
print(f'Recall Médio: {mean_recall:.4f}')

In [ ]:
#Definindo lista de hiperparâmetros a serem otimizados
param_grid_catboost = {
    'iterations': [5, 10, 20],
    'depth': [3, 4, 5],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 1.0],
    'colsample_bylevel': [0.8, 1.0]
}

#Criando o modelo CatBoost
catboost_model = CatBoostClassifier(random_state=42, verbose=False)

#Criando o objeto GridSearchCV com scoring='roc_auc'
grid_search_catboost = GridSearchCV(catboost_model, param_grid_catboost, cv=kf, scoring='roc_auc', verbose=1, n_jobs=-1)

#Realizando o grid search
grid_search_catboost.fit(X, y)

#Melhores hiperparâmetros encontrados
best_params_catboost = grid_search_catboost.best_params_
best_auc_catboost = grid_search_catboost.best_score_

# Imprima os melhores hiperparâmetro
print("Melhores Hiperparâmetros CatBoost:")
print(best_params_catboost)

7.1 CATBOOST com melhores hiperparâmetros

In [ ]:
#Definindo os hiperparâmetros
catboost_params = {
    'colsample_bylevel': 0.8,
    'depth': 3,
    'iterations': 5,
    'learning_rate': 0.2,
    'subsample': 1.0
}

#Criando o modelo CatBoost
catboost_model = CatBoostClassifier(**catboost_params, random_state=42, verbose=False)

#Definindo o número de folds
n_folds = 3

#Objeto de validação cruzada com 3 folds
kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)

#Realizando a validação cruzada e calculando as métricas
cross_val_accuracy_scores = cross_val_score(catboost_model, X, y, cv=kf, scoring='accuracy')
cross_val_auc_scores = cross_val_score(catboost_model, X, y, cv=kf, scoring='roc_auc')
cross_val_precision_scores = cross_val_score(catboost_model, X, y, cv=kf, scoring='precision')
cross_val_recall_scores = cross_val_score(catboost_model, X, y, cv=kf, scoring='recall')
cross_val_f1_scores = cross_val_score(catboost_model, X, y, cv=kf, scoring='f1')

#Pontuações de validação cruzada para cada fold
for i, (accuracy, auc, precision, recall) in enumerate(zip(cross_val_accuracy_scores, cross_val_auc_scores, cross_val_precision_scores, cross_val_recall_scores)):
    print(f'Fold {i + 1} - Acurácia: {accuracy:.4f}, AUC: {auc:.4f}, Precisão: {precision:.4f}, Recall: {recall:.4f}')

#Calculando as métricas médias em todos os folds
mean_accuracy = cross_val_accuracy_scores.mean()
mean_auc = cross_val_auc_scores.mean()
mean_precision = cross_val_precision_scores.mean()
mean_recall = cross_val_recall_scores.mean()
mean_f1 = cross_val_f1_scores.mean()

#Imprimindo as métricas médias
print("CATBOOST ajustado:")
print(f'Acurácia Média: {mean_accuracy:.4f}')
print(f'AUC Médio: {mean_auc:.4f}')
print(f'Precisão Média: {mean_precision:.4f}')
print(f'Recall Médio: {mean_recall:.4f}')
print(f'F1: {mean_f1:.4f}')

runModel(catboost_model, X, y, title="Catboost")

8.Variáveis selecionadas pelo algoritmo

Foi aplicado o BORUTA e SHAP para observar quais foram as principais variáveis utilizada pelo modelo para predição do desfecho pelo algoritmo CATBOOST

8.1 BORUTA

In [ ]:
#Hiperparâmetros do CatBoost
catboost_params = {'colsample_bylevel': 0.8, 'depth': 3, 'iterations': 5, 'learning_rate': 0.01, 'subsample': 0.8}

#Modelo CatBoost
catboost_model = CatBoostClassifier(**catboost_params)

#Definindo as variáveis independentes e a variável dependente
X = dfALDH2.drop(columns=['Mutação'])
y = dfALDH2['Mutação'].values

#Treinando o modelo CatBoost
catboost_model.fit(X, y)

#Importâncias das características do CatBoost
feature_importances = catboost_model.feature_importances_

#DataFrame com as importâncias das características
importance_df = pd.DataFrame({'Feature': X.columns, 'Importance': feature_importances})

#Ordeneando as características por importância
importance_df = importance_df.sort_values(by='Importance', ascending=False)

#Selecionando as características com importância maior que zero
selected_features = importance_df.loc[importance_df['Importance'] > 0, 'Feature'].tolist()

# Imprima ou utilize as características selecionadas
print("Características selecionadas pelo CatBoost:")
print(selected_features)


8.2 SHAP

In [ ]:
#Modelo CatBoost
catboost_model = CatBoostClassifier(**catboost_params)

#Treinando o modelo CatBoost
catboost_model.fit(X, y)

#Aplicando o explainer SHAP
explainer = shap.Explainer(catboost_model)

#Obtendo os valores SHAP para todos os pontos de dados
shap_values = explainer.shap_values(X)

#Resumo de plotagem para visualizar a importância das características
shap.summary_plot(shap_values, X)

# Criação do gráfico individual para pontos específicos
shap.force_plot(explainer.expected_value, shap_values[0, :], X.iloc[0, :])

# Inicializando o JavaScript para o SHAP
shap.initjs()

#Exibindo o force plot para um ponto específico
shap.force_plot(explainer.expected_value, shap_values[i, :], X.iloc[i, :])